In [368]:
import pandas as pd
import re

In [408]:
df = pd.read_csv('fundacurrentlistings20240517_dataset.csv')

In [409]:
df = df.rename(columns={'num_of_bathrooms':'num_of_floors'})
df.describe()

,url,price,address,descrip,zip_code,size,year,living_area,kind_of_house,building_type,num_of_rooms,num_of_floors,layout,energy_label,insulation,heating,ownership,parking,last_ask_price,city
count,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951
unique,951,250,922,844,593,161,170,174,59,3,31,29,605,51,56,40,43,18,357,1
top,https://www.funda.nl/huur/amsterdam/appartemen...,€ 2.750 /mnd,na,na,1043 BV Amsterdam,na,2024,na,Bovenwoning (appartement),Bestaande bouw,3 kamers (2 slaapkamers),1 woonlaag,na,na,na,na,na,na,na,amsterdam
freq,1,36,20,20,153,136,232,67,256,623,375,295,101,374,496,540,618,849,65,951


In [419]:
df = df.drop_duplicates()
df = df.drop(columns={'size', 'city', 'last_ask_price'})

# clean the price
df['price'] = df['price'].str.replace('€ ', '')
df['price'] = df['price'].str.replace(' /mnd', '')
df['price'] = df['price'].str.replace('k', '')
df['price'] = df['price'].str.replace('na', '0')
df['price'] = df['price'].str.replace('Huurprijs op aanvraag', '0')
df['price'] = df['price'].str.replace('.', '')

# drop all rows where dependent variable price is equal to zero and higher than 12500 euros a months. Those are buy listings listed wrongly
df = df.drop(df[df['price'] == '0'].index)
df = df.drop(df[pd.to_numeric(df['price']) > 12500].index)
# df.sort_values('price', ascending=False, key=pd.to_numeric)
df.describe()

,url,price,address,descrip,zip_code,year,living_area,kind_of_house,building_type,num_of_rooms,num_of_floors,layout,energy_label,insulation,heating,ownership,parking,last_ask_price
count,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893
unique,893,226,885,805,563,169,165,55,3,27,26,597,51,56,40,41,16,330
top,https://www.funda.nl/huur/amsterdam/appartemen...,2750,Reguliersgracht 15 3,NARITAWEG - HET HART VAN VOORUITZIEND AMSTE...,1043BV,2024,70,Bovenwoning (appartement),Bestaande bouw,3 kamers (2 slaapkamers),1 woonlaag,na,na,na,na,na,na,na
freq,1,36,2,6,153,232,60,255,588,373,295,51,319,441,485,563,794,45


In [415]:
# clean zip_code. Should be len = 6 with 4 numbers and 2 letters
df['zip_code'] = df['zip_code'].str[:7]
df['zip_code'] = df['zip_code'].str.replace(' ', '')

In [416]:
# remove non-houses
df = df.drop(df[df['kind_of_house'] == 'Garage'].index)
df = df.drop(df[df['kind_of_house'] == 'Inpandige garage'].index)
df = df.drop(df[df['kind_of_house'] == 'parkeerkelder'].index)
df = df.drop(df[df['kind_of_house'] == 'Parkeerplaats'].index)
df.describe()

,url,price,address,descrip,zip_code,size,year,living_area,kind_of_house,building_type,num_of_rooms,num_of_floors,layout,energy_label,insulation,heating,ownership,parking,last_ask_price,city
count,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893
unique,893,226,885,805,563,159,169,165,55,3,27,26,597,51,56,40,41,16,330,1
top,https://www.funda.nl/huur/amsterdam/appartemen...,2750,Reguliersgracht 15 3,NARITAWEG - HET HART VAN VOORUITZIEND AMSTE...,1043BV,na,2024,70 m²,Bovenwoning (appartement),Bestaande bouw,3 kamers (2 slaapkamers),1 woonlaag,na,na,na,na,na,na,na,amsterdam
freq,1,36,2,6,153,82,232,60,255,588,373,295,51,319,441,485,563,794,45,893


In [417]:
df['living_area'] = df['living_area'].str.replace(' m²', '')
df['living_area'] = df['living_area'].str.replace('na', '')

In [420]:
df.head()

,url,price,address,descrip,zip_code,year,living_area,kind_of_house,building_type,num_of_rooms,num_of_floors,layout,energy_label,insulation,heating,ownership,parking,last_ask_price
0,https://www.funda.nl/huur/amsterdam/appartemen...,1045,Schipluidenlaan 254,Podium * VERHUUR GESTART * 147 middenhuur ...,1062HE,2024,44,Portiekflat,Nieuwbouw,2 kamers (1 slaapkamer),6 woonlagen,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,na,na,na,na,na,€ 1.045 per maand (geen servicekosten)
1,https://www.funda.nl/huur/amsterdam/appartemen...,1068,Krijn Taconiskade 299,*** See English version below *** Huren D...,1087HW,2022,44,Portiekflat (appartement met open portiek),Nieuwbouw,1 kamer (1 slaapkamer),6 woonlagen,Aantal kamers1 kamer (1 slaapkamer)Aantal woon...,A++,Eco-bouw en volledig geïsoleerd,Gehele vloerverwarming,na,na,€ 1.068 per maand (exclusief servicekosten à €...
2,https://www.funda.nl/huur/amsterdam/appartemen...,1099,Haarlemmerweg,** BINNENKORT IN VERHUUR | WesterparkWest f...,1014BL,2024,50,Galerijflat,Nieuwbouw,2 kamers (1 slaapkamer),10 woonlagen,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,na,na,na,Nee,na,€ 1.099 per maand (geen servicekosten)
3,https://www.funda.nl/huur/amsterdam/appartemen...,1103,Krijn Taconiskade 441,*** See English version below *** Huren D...,1087HW,2022,44,Portiekflat (appartement met open portiek),Nieuwbouw,1 kamer (1 slaapkamer),6 woonlagen,Aantal kamers1 kamer (1 slaapkamer)Aantal woon...,A++,Eco-bouw en volledig geïsoleerd,Gehele vloerverwarming,na,na,€ 1.103 per maand (exclusief servicekosten à €...
4,https://www.funda.nl/huur/amsterdam/appartemen...,1105,Willem Frogerstraat 41,Podium *VERHUUR GESTART* 147 middenhuur app...,1062HZ,2024,56,Portiekflat,Nieuwbouw,2 kamers (1 slaapkamer),6 woonlagen,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,na,na,na,na,na,€ 1.105 per maand (geen servicekosten)
